In [2]:
import Pkg; 
Pkg.activate(@__DIR__); 
Pkg.instantiate()

  Activating environment at `~/Documents/PhD/courses/16745_optimal_control_rl/project/ballbot_ocrl_navigation/Project.toml`


In [3]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using ForwardDiff
using Plots
using RobotDynamics
using Ipopt
using FileIO, JLD2
using Colors

import Random
import ECOS
import Convex as cvx
import MathOptInterface as MOI
import DifferentiableCollisions as dc
import MeshCat as mc;

WebIO._IJuliaInit()

In [4]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","ballbot_model.jl"))

rk4 (generic function with 1 method)

## Load ballbot model

In [6]:
ballbot = Ballbot_arms()

Ballbot{StateCache{Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Prismatic{Float64}}}, Vector{Joint{Float64, Revolute{Float64}}}}, 2}}}(Spanning tree:
Vertex: world (root)
  Vertex: Link_Xtran, Edge: Joint_World_Xtran
    Vertex: Link_Ytran, Edge: Joint_World_Ytran
      Vertex: Link_Pitch, Edge: xAngle
        Vertex: Link_Roll, Edge: yAngle
          Vertex: Link_Yaw, Edge: yaw
            Vertex: RArm1, Edge: JRA1
              Vertex: RArm2, Edge: JRA2
                Vertex: RArm3, Edge: JRA3
                  Vertex: RArm4, Edge: JRA4
                    Vertex: RArm5, Edge: JRA5
                      Vertex: RArm6, Edge: JRA6
                        Vertex: RArm7, Edge: JRA7
            Vertex: LArm1, Edge: JLA1
              Vertex: LArm2, Edge: JLA2
                Vertex: LArm3, Edge: JLA3
                  Vertex: LArm4, Edge: JLA4
                    Vertex: LArm5, Edge: JLA5
                      Vertex: LArm6, Edge: JLA6
                 

In [7]:
state = ballbot.statecache[Float32]

MechanismState{Float32, Float64, Float64, …}(…)

In [8]:
world = RigidBodyDynamics.findbody(ballbot.mech, "world")

right_end = RigidBodyDynamics.findbody(ballbot.mech, "RArm4")
right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
r_end_eff_pos = transform(state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

left_end = RigidBodyDynamics.findbody(ballbot.mech, "LArm4")
left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
l_end_eff_pos = transform(state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))

@show r_end_eff_pos
@show l_end_eff_pos

r_end_eff_pos = Point3D in "world": [0.27577000521726047, -0.07634999635039783, 1.0399079977325498]
l_end_eff_pos = Point3D in "world": [-0.2757700113342824, -0.07634998445254329, 1.0399079915443918]


Point3D in "world": [-0.2757700113342824, -0.07634998445254329, 1.0399079915443918]

In [9]:
function get_endeffector_poses(model::Ballbot, q::AbstractVector{T}) where {T} 
    state = model.statecache[T]
    # Convert from state ordering to the ordering of the mechanism
    #set_configuration!(state, q)
    copyto!(state.q, q)
    r_end_eff_pos = transform(state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))
    l_end_eff_pos = transform(state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))
    return r_end_eff_pos.v, l_end_eff_pos.v
end

get_endeffector_poses (generic function with 1 method)

In [10]:
get_endeffector_poses(ballbot, zeros(19))

([0.27577000521726047, -0.07634999635039783, 1.0399079977325498], [-0.2757700113342824, -0.07634998445254329, 1.0399079915443918])

In [11]:
q = zeros(9)
q[1] = 0 # move to a distance
q[3] = 0 # lean forward
q[6] = 0.785 # right shoulder
q[7] = -0.785 # left shoulder
q[8] = 1.5708 # right elbow 90 deg
q[9] = -1.5708 # left elbow 90 deg

-1.5708

In [14]:
ree_pos, lee_pos = get_endeffector_poses(ballbot, q)

([0.27577000521726047, -0.07634999635039783, 1.0399079977325498], [-0.2757700113342824, -0.07634998445254329, 1.0399079915443918])

In [15]:
function momentum_function(model::Ballbot, q::AbstractVector{T}) where {T} 
    state = model.statecache[T]
    res = model.dyncache[T]
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state.q, q)
    #set_configuration!(state, q)
    out = momentum_matrix(state)
    return out.angular
end

momentum_function (generic function with 1 method)

In [16]:
function com_function(model::Ballbot, q::AbstractVector{T}) where {T} 
    state = model.statecache[T]
    res = model.dyncache[T]
    #out = model.momentum_m
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state.q, q)
    #set_configuration!(state, q)
    out = center_of_mass(state)
    return out.v
end

com_function (generic function with 1 method)

In [17]:
function skew(ω::Vector{T}) where {T}
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

skew (generic function with 1 method)

In [18]:
function create_idx(nq,N) 
    nq̇ = nq̈ = nq
    nh = nḣ = nr = nṙ = nr̈ = nF = nτ = 3
   
    # Our Z vector is [x0, u0, x1, u1, …, xN]
    nzi = nq + nq̇ + nq̈ + nr + nṙ + nr̈ + nh + nḣ + nF + nτ
    nz = N * nzi
    
    q = [(i - 1) * nzi .+ (1 : nq) for i = 1:N]
    q̇ = [(i - 1) * nzi .+ ((nq + 1) : (2*nq)) for i = 1:N]
    q̈ = [(i - 1) * nzi .+ ((2*nq + 1) : (3*nq)) for i = 1:N]
    r = [(i - 1) * nzi .+ ((3*nq + 1) : (3*nq + nr)) for i = 1:N]
    ṙ = [(i - 1) * nzi .+ ((3*nq + nr + 1) : (3*nq + 2*nr)) for i = 1:N]
    r̈ = [(i - 1) * nzi .+ ((3*nq + 2*nr + 1) : (3*nq + 3*nr)) for i = 1:N]
    h = [(i - 1) * nzi .+ ((3*nq + 3*nr + 1) : (3*nq + 3*nr + nh)) for i = 1:N]
    ḣ = [(i - 1) * nzi .+ ((3*nq + 3*nr + nh + 1) : (3*nq + 3*nr + 2*nh)) for i = 1:N]
    F = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 1) : (3*nq + 3*nr + 2*nh + nF)) for i = 1:N]
    τ = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + nF + 1) : (3*nq + 3*nr + 2*nh + nF + nτ)) for i = 1:N]
    
    # Constraint indexing when stacked up   
    nci = nr̈ + nh + nḣ + nh + nr + nṙ + nr + nq + nq̇
          
    dcr̈ = [(i - 1) * (nci) .+ (1 : nr̈) for i = 1:N]
    dch = [(i - 1) * (nci) .+ ((nr̈ + 1) : (nr̈ + nh)) for i = 1:N]
    dcḣ = [(i - 1) * (nci) .+ ((nr̈ + nh + 1) : (nr̈ + 2*nh)) for i = 1:N]
    jcr = [(i - 1) * (nci) .+ ((nr̈ + 2*nh + 1) : (2*nr̈ + 2*nh)) for i = 1:N]
    
    cch = [(i - 1) * (nci) .+ ((2*nr̈ + 2*nh + 1) : (2*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccr = [(i - 1) * (nci) .+ ((2*nr̈ + 3*nh + 1) : (3*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccṙ = [(i - 1) * (nci) .+ ((3*nr̈ + 3*nh + 1) : (4*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccq = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + 1) : (4*nr̈ + 3*nh + nq)) for i = 1:(N - 1)]
    ccq̇ = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + nq + 1) : (4*nr̈ + 3*nh + 2*nq)) for i = 1:(N - 1)]
    
    nciter = nci * (N - 1) + (nr + nr̈ + nh + nḣ)
    ncfix = nq + nq + nq̇ + nq̈ + nḣ + nh
    println(nciter)
    
    icq = nciter .+ (1 : nq)
    gcq = nciter .+ ((nq + 1) : (2*nq))
    icq̇ = nciter .+ ((2*nq + 1) : (3*nq))
    icq̈ = nciter .+ ((3*nq + 1) : (4*nq))
    gcḣ = nciter .+ ((4*nq + 1) : (4*nq + nh))
    gch = nciter .+ ((4*nq + nh + 1) : (4*nq + 2*nh))
    
    nc = nciter + ncfix
   
    return (q=q,q̇=q̇,q̈=q̈,r=r,ṙ=ṙ,r̈=r̈,h=h,ḣ=ḣ,F=F,τ=τ,
            dcr̈=dcr̈,dch=dch,dcḣ=dcḣ,cch=cch,ccr=ccr,ccṙ=ccṙ,jcr=jcr,ccq=ccq,ccq̇=ccq̇,
            icq=icq,gcq=gcq,icq̇=icq̇,icq̈=icq̈,gcḣ=gcḣ,
            nzi=nzi,N=N,nz=nz,nc=nc,nq=nq,nq̇=nq̇,nq̈=nq̈,nr=nr,nṙ=nṙ,nr̈=nr̈,nh=nh,nḣ=nḣ,nF=nF,nτ=nτ,gch=gch)
end

create_idx (generic function with 1 method)

In [19]:
function centroidal_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    qic, qgoal, q̇ic, q̈ic = params.qic, params.qgoal, params.q̇ic, params.q̈ic
    g, m = params.g, params.m
    model = params.model
    
    c = zeros(eltype(Z), idx.nc)
    #println("init")
    for i = 2:N
        #println("Configuration and derivatives")
        # Configuration and derivatives
        qi = Z[idx.q[i]]
        q̇i = Z[idx.q̇[i]]
        q̈i = Z[idx.q̈[i]]
        qimn = Z[idx.q[i-1]]
        q̇imn = Z[idx.q̇[i-1]]
        q̈imn = Z[idx.q̈[i-1]]
        #println("COM and derivatives")
        # COM and derivatives
        ri = Z[idx.r[i]]
        ṙi = Z[idx.ṙ[i]]
        r̈i = Z[idx.r̈[i]]
        rimn = Z[idx.r[i-1]]
        ṙimn = Z[idx.ṙ[i-1]]
        #println("Angular momentum and derivative")
        # Angular momentum and derivative
        hi = Z[idx.h[i]]
        ḣi = Z[idx.ḣ[i]]
        himn = Z[idx.h[i-1]]
        ḣimn = Z[idx.ḣ[i-1]]
        #println("Contact point, force and torque")
        # Contact point, force and torque
        ci = [qi[1:2]; 0]
        Fi = Z[idx.F[i]]
        τi = Z[idx.τ[i]] 
        
        #println("Centroidal dynamics")
        #println("N: ", i)
        # Centroidal dynamics
        #println("Centroidal dynamics: 1")
        c[idx.dcr̈[i]] .= Fi + m*g - m*r̈i
        #println("Centroidal dynamics: 2")
        c[idx.dcḣ[i]] .= skew(ci-ri)*Fi + τi - ḣi
        #println("Centroidal dynamics: 3")
        c[idx.dch[i]] .= momentum_function(model, qi)*q̇i - hi
        #println("Collocation constraints")
        
        # Collocation constraints
        c[idx.ccq[i-1]] .= qi - qimn - q̇i*dt
        c[idx.ccq̇[i-1]] .= q̇i - q̇imn - q̈i*dt
        c[idx.cch[i-1]] .= hi - himn - ḣi*dt
        c[idx.ccr[i-1]] .= ri - rimn - (ṙi + ṙimn)*dt/2
        c[idx.ccṙ[i-1]] .= ṙi - ṙimn - r̈i*dt
        #println("Joint configuration constraint")
        
        # Joint configuration constraint
        c[idx.jcr[i]] .= com_function(model, qi) - ri
    #println("Initial and goal constraints")
    
    c1 = [Z[idx.q[1]][1]; Z[idx.q[1]][2]; 0]
    c[idx.dcr̈[1]] .= Z[idx.F[1]] + m*g - m*Z[idx.r̈[1]]
    c[idx.dcḣ[1]] .= skew(c1-Z[idx.r[1]])*Z[idx.F[1]] + Z[idx.τ[1]] - Z[idx.ḣ[1]]
    c[idx.dch[1]] .= momentum_function(model, Z[idx.q[1]])*Z[idx.q̇[1]] - Z[idx.h[1]]
    c[idx.jcr[1]] .= com_function(model, Z[idx.q[1]]) - Z[idx.r[1]]
        
    # Initial and goal constraints
    c[idx.icq] .= qic - Z[idx.q[1]]
    c[idx.gcq] .= qgoal - Z[idx.q[end]] 
    c[idx.icq̇] .= q̇ic - Z[idx.q̇[1]]
    c[idx.icq̈] .= q̈ic - Z[idx.q̈[1]]
    c[idx.gch] .= Z[idx.h[end]]
    c[idx.gcḣ] .= Z[idx.ḣ[end]]
    end
    return c
end

centroidal_equality_constraints (generic function with 1 method)

In [20]:
function centroidal_inequality_constraint(params, Z)
    return zeros(eltype(Z), 0)
end

centroidal_inequality_constraint (generic function with 1 method)

In [21]:
ballbot.mech

Spanning tree:
Vertex: world (root)
  Vertex: Link_Xtran, Edge: Joint_World_Xtran
    Vertex: Link_Ytran, Edge: Joint_World_Ytran
      Vertex: Link_Pitch, Edge: xAngle
        Vertex: Link_Roll, Edge: yAngle
          Vertex: Link_Yaw, Edge: yaw
            Vertex: RArm1, Edge: JRA1
              Vertex: RArm2, Edge: JRA2
                Vertex: RArm3, Edge: JRA3
                  Vertex: RArm4, Edge: JRA4
                    Vertex: RArm5, Edge: JRA5
                      Vertex: RArm6, Edge: JRA6
                        Vertex: RArm7, Edge: JRA7
            Vertex: LArm1, Edge: JLA1
              Vertex: LArm2, Edge: JLA2
                Vertex: LArm3, Edge: JLA3
                  Vertex: LArm4, Edge: JLA4
                    Vertex: LArm5, Edge: JLA5
                      Vertex: LArm6, Edge: JLA6
                        Vertex: LArm7, Edge: JLA7
No non-tree joints.

## Start visualizer

In [ ]:
# Initialize visualizer
# Create visualizer for ballbot
#delete!(vis)
mvis = MechanismVisualizer(ballbot.mech, URDFVisuals(urdf))

## Trajectory optimization

## Move from point A to point B

In [22]:
# Optimization Parameters
h = 0.1 #10 Hz
nq = 19
Tfinal = 2 # final time
Nt = Int(Tfinal/h)+1   # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

# Indexing 
idx = create_idx(nq,Nt);

n_nlp = idx.nz
m_nlp = idx.nc

# Cost matrices
Qeer = 100*Diagonal(ones(idx.nr))
Qeel = 100*Diagonal(ones(idx.nr))
Qb = 100*Diagonal(ones(2))
Qcom = Diagonal(ones(2))
Qḣ = Diagonal(ones(idx.nḣ))
Qq̈ = 10*Diagonal(ones(idx.nq̈))

diff_type = :auto

1192


:auto

In [23]:
# Create warm start trajectory
qic = zeros(nq)
qgoal = zeros(nq)
q̇ic = zeros(nq) 
q̈ic = zeros(nq)

qgoal[1] = 0.1
qgoal[2] = 0.1

#set_configuration!(ballbot.statecache[Float32], qic)


ball_ref = range(qic[1:2], qgoal[1:2], length = Nt) 
#ee_right_ref = [0.6672199882477828, -0.058433993692510657, 1.3880790779692234] 
#ee_left_ref = [-0.667219994435301, -0.05843398960535944, 1.3880790907086575]

#ee_right_ref[1] = ee_right_ref[1] + qgoal[1]
#ee_right_ref[2] = ee_right_ref[2] + qgoal[2];

In [24]:
m = mass(ballbot.mech)
g = [0.0, 0.0, -9.81];

In [25]:
# Create params
params = (dt = h,
          N = Nt,
          idx = idx,
          g = g, 
          m = m,
          model = ballbot,
          Qeer = Qeer,
          Qeel = Qeel,
          Qb = Qb,
          Qcom = Qcom,
          Qḣ = Qḣ,
          Qq̈ = Qq̈,
          qic = qic,
          qgoal = qgoal,
          q̇ic = q̇ic, 
          q̈ic = q̈ic,
          ball_ref = ball_ref); 
#          ee_right_ref = ee_right_ref,
#          ee_left_ref = ee_left_ref);

In [26]:
# Primal bounds 
x_l = ones(idx.nz)
x_u = ones(idx.nz)

for i = 1:(Nt)
    x_l[idx.q[i]] .= -Inf*x_l[idx.q[i]]
    #x_l[idx.q[i]][3:4] .= -0.8
    x_u[idx.q[i]] .= Inf*x_u[idx.q[i]]
    #x_u[idx.q[i]][3:4] .= 0.8
end

# Inequality constraint bounds (this is what we do when we have no inequality constraints)
c_l = zeros(0)
c_u = zeros(0)

verbose=true

true

In [27]:
z0 = zeros(idx.nz)
for i in 1:(Nt)
    z0[idx.q[i]] .= [ball_ref[i]; zeros(17)]
end

In [28]:
function centroidal_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N 
    # Stage cost
    J = 0 
    for i = 1:(N)
        qi = Z[idx.q[i]]
        ri = Z[idx.r[i]]
        ḣi = Z[idx.ḣ[i]]
        q̈i = Z[idx.q̈[i]]
        J += stage_cost(params, qi, ri, ḣi, q̈i, i)
    end
    return J 
end

centroidal_cost (generic function with 1 method)

In [ ]:
function stage_cost(params,qi,ri,ḣi,q̈i,k)
    ball_ref = params.ball_ref
    #ee_right_ref, ee_left_ref = params.ee_right_ref, params.ee_left_ref
    Qeer, Qeel, Qb, Qcom, Qḣ, Qq̈ = params.Qeer, params.Qeel, params.Qb, params.Qcom, params.Qḣ, params.Qq̈
    model = params.model
    
    ps = qi[1:2] 
    ps_com = ri[1:2]
    #ree_pos, lee_pos = get_endeffector_poses(model, qi)
    return 0.5*((ps-ball_ref[k])'*Qb*(ps-ball_ref[k])) + 0.5*((ps-ps_com)'*Qcom*(ps-ps_com)) 
         + 0.5*(ḣi'*Qḣ*ḣi) + 0.5*(q̈i'*Qq̈*q̈i) #+ 0.5*(ree_pos'*Qeer*ree_pos) + 0.5*(lee_pos'*Qeel*lee_pos)
end

In [ ]:
Z = fmincon(centroidal_cost,centroidal_equality_constraints,centroidal_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose);

In [ ]:
#FileIO.save("utils/ballbot_centroidal_z.jld2","centroidal_z", Z)

In [ ]:
qsol = [Z[idx.q[i]] for i = 1:Nt];

In [ ]:
q̇sol = [Z[idx.q̇[i]] for i = 1:Nt];

In [ ]:
qm = hcat(qsol...)
q̇m = hcat(q̇sol...);

In [ ]:
plot(thist,qm[1,:])
plot!(thist,qm[2,:])

In [ ]:
plot(thist,qm[3,:])
plot!(thist,qm[4,:])

In [ ]:
plot(thist,qm[5,:])

In [ ]:
plot(thist,q̇m[1,:])
plot!(thist,q̇m[2,:])

In [ ]:
plot(thist,q̇m[3,:])
plot!(thist,q̇m[4,:])

In [ ]:
plot(thist,q̇m[5,:])

In [ ]:
qsim = [SVector{19}(qi) for qi in qsol];

In [ ]:
# Uncomment for upload animation to visualizer
animation = mc.Animation(mvis, thist, qsim)
setanimation!(mvis, animation)

In [ ]:
mvis